# Figure 2 — Heatmap of Noise-Shaped Synchrony

This notebook reproduces **Figure 2** of the manuscript  
*"Noise-shaped Synchrony in Neuronal Oscillator Networks"*.

It visualizes how synchrony depends on system parameters using a **heatmap representation** based on precomputed data.

---

## What this notebook does

- Loads precomputed parameter-scan results from CSV files  
- Constructs a 2D heatmap of synchrony metrics  
- Generates the final heatmap figure shown in the manuscript

This notebook **does not run large-scale simulations** and is therefore safe and fast to execute.

---

## Input data

The following data files (located in this folder) are used as inputs:

- Parameter scan CSV files generated offline (see files in this directory)

---

## Output

Running all cells will generate one or more heatmap figure files corresponding to **Figure 2** in the manuscript.

---

## How to run

Simply execute all cells from top to bottom:

- Jupyter menu: **Kernel → Restart & Run All**
- Or execute cells sequentially

Typical runtime on a standard laptop: **a few seconds**.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.transforms as mtransforms

In [ ]:
def plot_labels_heatmap(labels_df, output_file=None, use_tex=False, cluster_colors=None, ymin=None, ymax=None, n_labels=7, random_seed=42, circle_size=6.0, label_size=11.0, label_xoff=0.60, label_yoff=0.20, label_pad=0.25):
    col_map = {c.lower(): c for c in labels_df.columns}
    for req in ("noise", "coupling", "cluster"):
        if req not in col_map:
            raise ValueError(f"labels_df must contain '{req}' column.")
    ncol, ccol, lcol = col_map["noise"], col_map["coupling"], col_map["cluster"]
    noise_vals = labels_df[ncol].astype(float).to_numpy()
    coupling_vals = labels_df[ccol].astype(float).to_numpy()
    cluster_vals = labels_df[lcol].astype(int).to_numpy()
    if ymin is not None or ymax is not None:
        mask = np.ones_like(coupling_vals, dtype=bool)
        if ymin is not None:
            mask &= coupling_vals >= ymin
        if ymax is not None:
            mask &= coupling_vals <= ymax
        noise_vals, coupling_vals, cluster_vals = noise_vals[mask], coupling_vals[mask], cluster_vals[mask]
    noise_unique = np.unique(noise_vals)
    coupling_unique = np.unique(coupling_vals)
    heatmap_data = np.full((len(coupling_unique), len(noise_unique)), np.nan)
    noise_idx = {v: j for j, v in enumerate(noise_unique)}
    coupling_idx = {v: i for i, v in enumerate(coupling_unique)}
    for n, c, cl in zip(noise_vals, coupling_vals, cluster_vals):
        heatmap_data[coupling_idx[c], noise_idx[n]] = cl
    min_cl = int(np.nanmin(cluster_vals))
    max_cl = int(np.nanmax(cluster_vals))
    classes = np.arange(min_cl, max_cl + 1)
    if cluster_colors is not None:
        colors = [cluster_colors.get(int(cl), "gray") for cl in classes]
        cluster_cmap = ListedColormap(colors)
    else:
        cmap_base = plt.get_cmap("tab20", len(classes) if len(classes) <= 20 else len(classes))
        cluster_cmap = ListedColormap(getattr(cmap_base, "colors", [cmap_base(i) for i in range(len(classes))]))
    boundaries = np.arange(min_cl - 0.5, max_cl + 1.5, 1.0)
    norm = BoundaryNorm(boundaries, ncolors=cluster_cmap.N)
    plt.rcParams.update({"text.usetex": use_tex,"font.family": "serif","axes.linewidth": 1.2,"xtick.direction": "in","ytick.direction": "in","xtick.major.size": 6,"ytick.major.size": 6,"xtick.minor.size": 3,"ytick.minor.size": 3,"legend.frameon": True,"legend.edgecolor": "black"})
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(10, 6))
    hm = sns.heatmap(heatmap_data, ax=ax, xticklabels=np.round(noise_unique, 4), yticklabels=np.round(coupling_unique, 4), cmap=cluster_cmap, norm=norm, cbar_kws={'pad': 0.01, 'ticks': classes}, linewidths=0.1, linecolor='gray')
    desired_vals = np.array([0.0, 0.0002, 0.0006, 0.001, 0.0014, 0.0018, 0.0025, 0.0035, 0.0045, 0.0055, 0.0065, 0.0075, 0.0085, 0.0095, 0.011, 0.013, 0.015, 0.017, 0.019, 0.025, 0.04, 0.06, 0.08, 0.1, 0.3, 0.5])
    desired_labels = ['0.0000', '0.0002', '0.0006', '0.0010', '0.0014', '0.0018', '0.0025', '0.0035', '0.0045', '0.0055', '0.0065', '0.0075', '0.0085', '0.0095', '0.0110', '0.0130', '0.0150', '0.0170', '0.0190', '0.0250', '0.0400', '0.0600', '0.0800', '0.1000', '0.3000', '0.5000']
    xticks_pos, xticks_lab = [], []
    for v, lab in zip(desired_vals, desired_labels):
        hits = np.where(np.isclose(noise_unique, v, atol=1e-12))[0]
        if len(hits):
            j = int(hits[0])
            xticks_pos.append(j + 0.5)
            xticks_lab.append(lab)
    if xticks_pos:
        ax.set_xticks(xticks_pos)
        ax.set_xticklabels(xticks_lab, rotation=90, ha='center')
        ax.set_xlim(0, len(noise_unique))
    desired_y = np.round(np.arange(0.00, 0.1001, 0.01), 2)
    idxs = [i for i, v in enumerate(coupling_unique) if np.any(np.isclose(v, desired_y, atol=1e-12))]
    ax.set_yticks([i + 0.5 for i in idxs])
    ax.set_yticklabels([f"{coupling_unique[i]:.2f}" for i in idxs], rotation=0, va='center')
    plt.tight_layout()
    ax.set_xlabel(r'$D$', fontsize=23)
    ax.set_ylabel(r'$C$', fontsize=23, rotation=0)
    Label_Size=16
    ax.tick_params(axis='both', which='major', labelsize=Label_Size)
    dy_in = -2 / 25.4
    offset = mtransforms.ScaledTranslation(0, dy_in, fig.dpi_scale_trans)
    ax.yaxis.get_label().set_transform(ax.yaxis.get_label().get_transform() + offset)
    ax.invert_yaxis()
    target_val = 0.06
    tol = 1e-12
    hits = np.where(np.isclose(coupling_unique, target_val, atol=tol))[0]
    if len(hits):
        i = hits[0]
        y0, y1 = i, i + 1
        ax.hlines([y0, y1], xmin=0, xmax=len(noise_unique), colors='black', linestyles='--', alpha=0.6)
        ax.vlines([0, len(noise_unique)], ymin=y0, ymax=y1, colors='black', linestyles='--', alpha=0.6)
    cbar = hm.collections[0].colorbar
    cbar.set_label('')
    labels_map = {0: 'PN', 1: 'QS', 2: 'NS', 3: 'CS', 4: 'IS'}
    cbar.set_ticks(classes)
    if use_tex:
        cbar.set_ticklabels([r'$\mathrm{' + labels_map.get(int(cl), str(int(cl))) + '}$' for cl in classes])
        cbar.ax.tick_params(labelsize=18)
        for text in cbar.ax.get_yticklabels():
            text.set_family('serif')
    else:
        cbar.set_ticklabels([labels_map.get(int(cl), str(int(cl))) for cl in classes])
    D_points = [0.0002, 0.0013, 0.0037, 0.0080, 0.0130, 0.0200, 0.2000]
    row_hits = np.where(np.isclose(coupling_unique, 0.06, atol=1e-9))[0]
    if len(row_hits):
        i_row = int(row_hits[0])
    else:
        i_row = int(np.argmin(np.abs(coupling_unique - 0.06)))
    def nearest_index(val, sorted_array):
        return int(np.argmin(np.abs(sorted_array - val)))
    up_1mm = mtransforms.ScaledTranslation(0, 1.0/25.4, fig.dpi_scale_trans)
    text_transform = ax.transData + up_1mm
    for k, D_val in enumerate(D_points, start=1):
        j = nearest_index(D_val, noise_unique)
        x, y = j + 0.5, i_row + 0.5
        ax.text(x - float(label_xoff), y - float(label_yoff), f"{k}", ha="center", va="center", fontsize=label_size, zorder=11, transform=text_transform, bbox=dict(boxstyle=f"circle,pad={label_pad}", facecolor="white", edgecolor="black", linewidth=1.5))
    if output_file:
        plt.savefig(output_file, dpi=600, bbox_inches='tight')
    return fig, ax

In [ ]:
CSV_PATH = "clusters_B_3_k5.csv"
OUTPUT_FILE = "s_B_p_3_k5_no_arrows.png"
CLUSTER_COLORS = {0: "#e41a1c", 1: "#377eb8", 2: "#4daf4a", 3: "#984ea3", 4: "#ff7f00"}
df = pd.read_csv(CSV_PATH)
fig, ax = plot_labels_heatmap(df, output_file=None, use_tex=True, cluster_colors=CLUSTER_COLORS, ymin=0.0, ymax=0.1)
plt.tight_layout()
dy_in = 3 / 25.4
offset = mtransforms.ScaledTranslation(0, dy_in, fig.dpi_scale_trans)
ax.yaxis.get_label().set_transform(ax.yaxis.get_label().get_transform() + offset)
plt.savefig(OUTPUT_FILE, dpi=300, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_FILE}")